In [1]:
import numpy as np
import PIL
import random
from PIL import Image
from PIL import ImageDraw
import pandas as pd

## First Create a function that has two main inputs, a list of tuples called start_points (coordinate, connections)
## the second input should be a list (ideally sorted in some way) of points of size sum(connections)
## Input image is a PIL Image object whose size allows for the inclusion of the points givenin start_points and end_points
## Draw lines between these points in the given image
def connect_bolts(start_points, end_points, image, color, line_width):
    draw = ImageDraw.Draw(image)
    for point in start_points:
        #point is a tuple (coordinate, connections)
        for i in range(point[1]):
            #Line inputs are [pt_1,pt_2], fill, width
            draw.line([point[0], end_points[0]], fill = color, width = line_width)
            end_points = end_points[1:]

#Take an input of a list coordinates and determine how many connections it will have based on a input probability list
# return a list containing tuples (coordinate, num_connections)
def split_decider(coordinates, probability):
    start_pts = []
    for coord in coordinates:
        num_connections = random.choice(probability)
        start_pts.append((coord, num_connections))
    return start_pts

# Now something a little more specific to my goals: create a list of points a specific x-distance away, organized by their y-value
#  smallest to largest where the number of points created is the sum of the number of connections from my start_pt list
#    y_range is a tuple (begin, end)
def create_end_pts(start_pts, x_step, y_range):
    num_pts = 0
    x = start_pts[0][0][0] + int(x_step)
    end_pts = []
    for coord_num in start_pts:
        num_pts += coord_num[1]
    y_coords = []
    for i in range(num_pts):
        y = random.randint(y_range[0], y_range[1])
        y_coords.append(y)
    y_coords.sort()
    for y in y_coords:
        end_pts.append((x,y))
    return end_pts

## Lets put it all together with creating an image, with a number of iterations, an initial width, height and all the needed inputs for
##   for the previously made functions
def lightning_bolt_1(height, init_width, iterations, color, probability, fractional_diminish):
    #Determine the width of the image
    image_width = int(init_width)
    temp_width = int(init_width)
    for i in range(iterations):
        temp_width = int(temp_width*fractional_diminish)
        image_width += temp_width
    #Initiate Image
    image = Image.new("RGBA", (image_width,height))
    
    #Starting point
    start_pts = [(0, int(height/2))]
    
    # Initial values
    line_width = int(iterations)
    x_step = int(init_width)
    y_range = (0, image.height)
    color = (255,255,255,255)
    
    #Where the lightning happens
    for i in range(iterations+1):
        split_form = split_decider(start_pts,probability)
        end_pts = create_end_pts(split_form, x_step, y_range)
        connect_bolts(split_form, end_pts, image, color, line_width)
        start_pts = end_pts
        line_width = line_width-1
        x_step = int(x_step*fractional_diminish)
    return image